In [1]:
# importações
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from CONSTANTES import *

In [2]:
# teste:
for rede in redes_sociais.keys():

    ######################################
    # lidando com a rede social
    ######################################
    print('Rede Social --', rede)

    G = nx.read_edgelist("redes/" + redes_sociais[rede], comments='%', nodetype=int)
    grau_medio = np.array(G.degree)[:, 1].mean()
    N = len(G)
    print('<k>:', grau_medio, '\tN:', N)

Rede Social -- twitter
<k>: 2.809670517757809 	N: 23370


In [3]:
# teste de simulação:
aux = nx.erdos_renyi_graph(N, grau_medio/(N-1), seed=42)
len(aux)

23370